In [1]:
%matplotlib

Using matplotlib backend: agg


# 训练一个分类器
上一讲中已经看到如何去定义一个神经网络，计算损失值和更新网络的权重。你现在可能在想下一步。

## 关于数据？
一般情况下处理图像、文本、音频数据时，可以使用标准的Python包来加载数据到一个numpy数组中。然后把这个数组转换成torcvh.*Tensor。
* 图像可以使用Pilolow，OpenCV
* 音频可以使用scipy，librosa
* 文本可以使用原始Python和Cython来加载，或者使用NLTK或SpaCyc处理

特别的，对于图像任务，我们创建了一个包torchvision,它包含了处理一些基本图像数据集的方法。这些数据集包括Imagenet、CIFAR10、MNIST等。除了数据加载以外，torchvision还包含了图像转换器，torchvision.datasets和torch.utils.data.DataLoader。  
torchvision包不仅提供了巨大的便利，也避免了代码的重复。
在这个教程中，我们使用CIFAR10数据集，它有如下10个类别 ：‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’。CIFAR-10的图像都是 3x32x32大小的，即，3颜色通道，32x32像素。
![图片加载失败，请稍后重试](datasets.jpg "")
## 训练一个图像分类器
依次按照下列顺序进行：
1.使用torchvision加载和归一化CIFAR10训练集和测试机  
2.定义一个卷积神经网络  
3.定义损失函数  
4.在训练集上训练网络  
5.在测试集上测试网络  
## 1.读取和归一化 CIFAR10
使用torchvision可以非常容易地加载CIFAR10。

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

torchvision的输出是[0,1]的PILImage图像，我们把它转换为归一化范围为[-1，1]的张量

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='',train = True,
                                       download = False, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='', train=False,
                                       download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

* Tips：这里在CIFAR10官网已经提前下载好了数据集，和该文件放在了同一文件下，如电脑中没有CIFAR10数据集，则需将download参数置为True，root不指定参数  
我们展示一些训练图像。

In [4]:
import matplotlib.pyplot as plt
import numpy as np

#展示图像的函数

def imshow(img):
    img =img / 2 + 0.5 # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    
    
# 获取随机数据
dataiter = iter(trainloader)
images, labels = dataiter.next()

# 展示图象
imshow(torchvision.utils.make_grid(images))
# 显示图像标签
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

  cat  ship  frog  bird


## 2.定义一个卷积神经网络
从之前的神经网络一节复制神经网络代码，并修改为输入3通道图像。

In [5]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

## 3.定义损失函数和优化器
我们使用交叉熵作为损失函数，使用带动量的随机梯度下降。

In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## 4.训练网络
有趣的时刻开始了。我们只需在数据迭代器上循环，将数据输入给网络，并优化。

In [8]:
device = torch.device("cuda:0")      

for epoch in range(2): # 多批次循环
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # 获取输入
        inputs, labels = data
    
        # 梯度置0
        optimizer.zero_grad()
    
        # 正向传播，反向传播，优化
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
        #打印状态信息
        running_loss += loss.item()
        if i % 2000 == 1999: # 每2000批次打印一次
            print('[%d, %5d] loss: %.3f' %
                 (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        
print("Finished Training")

[1,  2000] loss: 2.058
[1,  4000] loss: 1.772
[1,  6000] loss: 1.639
[1,  8000] loss: 1.547
[1, 10000] loss: 1.485
[1, 12000] loss: 1.442
[2,  2000] loss: 1.371
[2,  4000] loss: 1.357
[2,  6000] loss: 1.310
[2,  8000] loss: 1.315
[2, 10000] loss: 1.281
[2, 12000] loss: 1.260
Finished Training


## 5.在测试集上测试网络
我们在整个训练集上进行了两次训练，但是我们需要检查网络是否从数据集中学习到有用的东西。通过预测神经网络输出的类标签与实际情况标签进行对比来进行检测。如果预测正确，我们把该样本添加到正确预测列表。第一步，显示测试集中图片并熟悉图片内容。

In [9]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# 显示图片
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

GroundTruth:    cat  ship  ship plane


让我们看看神经网络认为以上图片是什么。

In [10]:
output = net(images)

输出是10个标签的能量。 一个类别的能量越大，神经网络越认为它是这个类别。所以让我们得到最高能量的标签。

In [11]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

Predicted:  horse  ship   dog   cat


结果看来不错。

接下来让看看网络在整个测试集上的结果如何。

In [12]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 55 %


结果看起来不错，至少比随机选择要好，随机选择的正确率为10%。 似乎网络学习到了一些东西。

在识别哪一个类的时候好，哪一个不好呢？

In [13]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 63 %
Accuracy of   car : 74 %
Accuracy of  bird : 17 %
Accuracy of   cat : 26 %
Accuracy of  deer : 50 %
Accuracy of   dog : 49 %
Accuracy of  frog : 70 %
Accuracy of horse : 75 %
Accuracy of  ship : 68 %
Accuracy of truck : 55 %


下一步？
我们如何在GPU上运行神经网络呢？
## 在GPU上训练
把一个神经网络移动到GPU上训练就像把一个Tensor转换GPU上一样简单。并且这个操作会递归遍历有所模块，并将其参数和缓冲区转换为CUDA张量。

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 确认我们的电脑支持CUDA,然后显示CUDA信息

print(device)

cuda:0


本节的其余部分假定device是CUDA设备。

然后这些方法将递归遍历所有模块并将模块的参数和缓冲区 转换成CUDA张量：

net.to(device)
记住：inputs, targets 和 images 也要转换。

inputs, labels = inputs.to(device), labels.to(device)
为什么我们没注意到GPU的速度提升很多？那是因为网络非常的小。

实践: 尝试增加你的网络的宽度（第一个nn.Conv2d的第2个参数，第二个nn.Conv2d的第一个参数，它们需要是相同的数字），看看你得到了什么样的加速。
### 实现的目标：
* 深入了解了Pytorch的张量库和神经网络
* 训练了一个小网络来分类图片

## 多GPU训练
如果你想使用所有的GPU得到更大的加速， 请查看数据并行处理。